In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset=pd.read_csv('PP_Mtrls_and_their_Mchncl_Prprts.csv')
dataset.head()

,Material,Su,Sy,E,G,mu,Ro,Use
0,ANSI Steel SAE 1015 asrolled,421,314,207000,79000,0.3,7860,True
1,ANSI Steel SAE 1015 normalized,424,324,207000,79000,0.3,7860,True
2,ANSI Steel SAE 1015 annealed,386,284,207000,79000,0.3,7860,True
3,ANSI Steel SAE 1020 asrolled,448,331,207000,79000,0.3,7860,True
4,ANSI Steel SAE 1020 normalized,441,346,207000,79000,0.3,7860,True


In [3]:
dataset.drop('Material', axis=1, inplace=True)

In [4]:
columns_to_drop = ['Sy','mu']
dataset = dataset.drop(columns=columns_to_drop)
dataset.head()

,Su,E,G,Ro,Use
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [5]:
dataset['Use'] = dataset['Use'].astype('object')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Su      802 non-null    int64 
 1   E       802 non-null    int64 
 2   G       802 non-null    int64 
 3   Ro      802 non-null    int64 
 4   Use     802 non-null    object
dtypes: int64(4), object(1)
memory usage: 31.5+ KB


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

dataset.head()

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [7]:
label_encoder = LabelEncoder()

# Fit and transform the target column
dataset['Use_True'] = label_encoder.fit_transform(dataset['Use_True'])
dataset

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,1
1,424,207000,79000,7860,1
2,386,207000,79000,7860,1
3,448,207000,79000,7860,1
4,441,207000,79000,7860,1
...,...,...,...,...,...
797,980,206000,80000,7860,0
798,1100,206000,80000,7860,0
799,1200,206000,80000,7860,0
800,1030,206000,80000,7860,0


In [8]:
indep=dataset[['Su','E','G','Ro']]
dep=dataset["Use_True"]

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[254   1]
 [ 13   0]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       255
           1       0.00      0.00      0.00        13

    accuracy                           0.95       268
   macro avg       0.48      0.50      0.49       268
weighted avg       0.91      0.95      0.93       268



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.9259735803739921


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
print("The roc_auc_score is:",roc_auc_score)

The roc_auc_score is: 0.9714932126696832


In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005836,0.003496,0.001087,0.000548,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.930466,0.925773,0.916697,0.912005,0.929815,0.922951,0.007355,2
1,0.005374,0.003627,0.001296,0.000523,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.930466,0.925773,0.916697,0.912005,0.929815,0.922951,0.007355,2
2,0.001147,0.000599,0.001182,0.001212,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.930466,0.930466,0.916697,0.916697,0.929815,0.924828,0.006643,1
3,0.002757,0.001756,0.001198,0.000722,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.930466,0.925773,0.916697,0.912005,0.929815,0.922951,0.007355,2


In [18]:
su_input=float(input("Ultimate Tensile Strength (Su) in MPa:"))
E_input=float(input("Elastic Modulus (E) in MPa:"))
g_input=float(input("Shear Modulus (G) in MPa:"))
ro_input=float(input("Density (Ro) in Kg/m3:"))

Ultimate Tensile Strength (Su) in MPa:1000
Elastic Modulus (E) in MPa:119876
Shear Modulus (G) in MPa:67543
Density (Ro) in Kg/m3:8912


In [19]:
Future_Prediction=grid.predict([[su_input,E_input,g_input,ro_input]])
y=("Future_Prediction={}".format(Future_Prediction))
if (y==0):
    print("Future_Prediction=[False]")
else:
    print('Future_Prediction=[True]')

Future_Prediction=[True]
[CV 5/5] END ......penalty=l2, solver=newton-cg;, score=0.930 total time=   0.0s
[CV 4/5] END ..........penalty=l2, solver=lbfgs;, score=0.912 total time=   0.0s
[CV 5/5] END ..........penalty=l2, solver=lbfgs;, score=0.930 total time=   0.0s
[CV 2/5] END ......penalty=l2, solver=liblinear;, score=0.930 total time=   0.0s
[CV 1/5] END ...........penalty=l2, solver=saga;, score=0.930 total time=   0.0s
[CV 5/5] END ...........penalty=l2, solver=saga;, score=0.930 total time=   0.0s
[CV 1/5] END ......penalty=l2, solver=newton-cg;, score=0.930 total time=   0.0s
[CV 1/5] END ......penalty=l2, solver=liblinear;, score=0.930 total time=   0.0s
[CV 3/5] END ......penalty=l2, solver=liblinear;, score=0.917 total time=   0.0s
[CV 4/5] END ......penalty=l2, solver=liblinear;, score=0.917 total time=   0.0s
[CV 5/5] END ......penalty=l2, solver=liblinear;, score=0.930 total time=   0.0s
[CV 2/5] END ...........penalty=l2, solver=saga;, score=0.926 total time=   0.0s
[CV